***
# Lab 2 ML7331
> ### By Allen Hoskins, Brittany Lewandowski, Alexy Morris, Fidel Tamares


***

# Data Preparation
***

In [1]:
#import packages
import pandas as pd
import numpy as np
import lightgbm as lgb

import seaborn as sns
import sklearn as sk
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,SGDClassifier,LogisticRegressionCV
from sklearn.feature_selection import SelectPercentile,f_regression
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import time
import warnings


#allen Packages
from catboost import CatBoostClassifier

In [2]:
df = pd.read_csv(r'/Users/allen/Desktop/MSDS/ML1/archive/airline_merged.csv',low_memory=False)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
#removing unwanted column from previous export
df = df.drop(columns =['Unnamed: 0'],axis =1)

In [5]:
#dropping highly correlated columns
#https://www.codegrepper.com/code-examples/python/how+to+drop+highly+correlated+features
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

df = df.drop(columns = to_drop, axis=1)

In [6]:
#looking at dataframe after removing highly correlated fields.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 37 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   year                   int64  
 1   month                  int64  
 2   day                    int64  
 3   day_of_week            int64  
 4   airline_cd             object 
 5   airline_name           object 
 6   flight_number          int64  
 7   tail_number            object 
 8   origin_airport         object 
 9   airport_origin         object 
 10  city_origin            object 
 11  state_origin           object 
 12  country_origin         object 
 13  latitude_origin        float64
 14  longitude_origin       float64
 15  destination_airport    object 
 16  airport_destination    object 
 17  city_destination       object 
 18  state_destination      object 
 19  country_destination    object 
 20  latitude_destination   float64
 21  longitude_destination  float64
 22  scheduled_departur

In [7]:
#selecting subset of data set
airports = ['BHM']

In [8]:
df_air = df.loc[df['origin_airport'].isin(airports)]

In [9]:
df_air.describe()

,year,month,day,day_of_week,flight_number,latitude_origin,longitude_origin,latitude_destination,longitude_destination,scheduled_departure,departure_delay,taxi_out,scheduled_time,wheels_on,taxi_in,scheduled_arrival,diverted,cancelled,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
count,12514.0,12514.000000,12514.000000,12514.000000,12514.000000,1.251400e+04,1.251400e+04,12514.000000,12514.000000,12514.000000,12338.000000,12334.000000,12514.000000,12324.000000,12324.000000,12514.000000,12514.000000,12514.000000,2128.000000,2128.000000,2128.000000,2128.000000,2128.000000
mean,2015.0,6.150951,15.687710,3.879575,2876.476346,3.356294e+01,-8.675355e+01,34.252718,-88.175001,1192.116909,9.032906,13.783525,108.121864,1399.118306,8.632587,1416.011507,0.004075,0.014464,14.805921,0.091635,20.292293,27.732613,2.580827
std,0.0,3.384375,8.763785,1.992624,1951.952017,7.787860e-12,1.647104e-11,4.990106,8.325266,445.126945,43.869223,7.389709,36.974468,458.707388,5.844535,443.946518,0.063711,0.119397,32.820428,3.167858,73.036267,47.048389,16.190917
min,2015.0,1.000000,1.000000,1.000000,3.000000,3.356294e+01,-8.675355e+01,25.793250,-115.152330,500.000000,-20.000000,4.000000,52.000000,3.000000,1.000000,707.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.0,3.000000,8.000000,2.000000,1051.000000,3.356294e+01,-8.675355e+01,29.980470,-95.339720,745.000000,-5.000000,9.000000,76.000000,952.000000,5.000000,1009.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2015.0,6.000000,16.000000,4.000000,2525.000000,3.356294e+01,-8.675355e+01,33.640440,-84.426940,1200.000000,-2.000000,12.000000,113.000000,1402.000000,7.000000,1414.000000,0.000000,0.000000,3.000000,0.000000,0.000000,9.000000,0.000000
75%,2015.0,9.000000,23.000000,5.000000,4592.000000,3.356294e+01,-8.675355e+01,39.175400,-83.348840,1609.000000,4.000000,16.000000,123.000000,1823.000000,10.000000,1825.000000,0.000000,0.000000,18.000000,0.000000,10.000000,34.000000,0.000000
max,2015.0,12.000000,31.000000,7.000000,6813.000000,3.356294e+01,-8.675355e+01,42.212060,-73.872610,2055.000000,1988.000000,116.000000,260.000000,2357.000000,150.000000,2325.000000,1.000000,1.000000,493.000000,144.000000,1971.000000,420.000000,240.000000


In [10]:
#selecting columns that are dtype != numeric
non_numeric = df_air.select_dtypes(exclude = np.number).columns

In [16]:
#create binary response for logistic regression
# create a function
def is_delayed(delay,axis=1):
    if delay <= 0:
        return 0
    else:
        return 1
    
# create a new column based on condition
df_air['is_delay'] = df_air.loc[:,'departure_delay'].apply(is_delayed)

/var/folders/cn/_lgzxbk177j14_b82nbl10br0000gn/T/ipykernel_90970/3939397851.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_air['is_delay'] = df_air.loc[:,'departure_delay'].apply(is_delayed)


In [17]:
df_air.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12514 entries, 321 to 5817010
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   12514 non-null  int64  
 1   month                  12514 non-null  int64  
 2   day                    12514 non-null  int64  
 3   day_of_week            12514 non-null  int64  
 4   airline_cd             12514 non-null  object 
 5   airline_name           12514 non-null  object 
 6   flight_number          12514 non-null  int64  
 7   tail_number            12510 non-null  object 
 8   origin_airport         12514 non-null  object 
 9   airport_origin         12514 non-null  object 
 10  city_origin            12514 non-null  object 
 11  state_origin           12514 non-null  object 
 12  country_origin         12514 non-null  object 
 13  latitude_origin        12514 non-null  float64
 14  longitude_origin       12514 non-null  float64
 15

In [13]:
#Departure Delay - response
#altered from https://www.statology.org/one-hot-encoding-in-python/

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore',drop='first')

#perform one-hot encoding on non-numeric column s
encoder_df = pd.DataFrame(encoder.fit_transform(df_air[non_numeric]).toarray())

#return column names from encoder
encoder_df.columns = encoder.get_feature_names_out()

#merge one-hot encoded columns back with original DataFrame
encoded = df_air.join(encoder_df)

In [14]:
#drop original columns
final_df = encoded.drop(non_numeric, axis=1)

In [15]:
#remove target variable and column target was created from training data set
target_delay_cols = ['is_delay','departure_delay']
feature_delay_cols = final_df.loc[:, ~final_df.columns.isin(target_delay_cols)].columns
X_delay = final_df[feature_delay_cols] # Features
y_delay = final_df.is_delay # Target variable

AttributeError: 'DataFrame' object has no attribute 'is_delay'

In [ ]:
#remove target variable and column target was created from training data set
target_cancel_cols = ['cancelled','cancellation_reason']
feature_cancel_cols = final_df.loc[:, ~final_df.columns.isin(target_cancel_cols)].columns
X_cancel = final_df[feature_cancel_cols] # Features
y_cancel = final_df.is_delay # Target variable

In [ ]:
#split original dataset into features and target variable
X_delay_train,X_delay_test,y_delay_train,y_delay_test=train_test_split(X_delay,y_delay,test_size=0.20,random_state=0)

In [ ]:
#split original dataset into features and target variable
X_cancel_train,X_cancel_test,y_cancel_train,y_cancel_test=train_test_split(X_cancel,y_cancel,test_size=0.20,random_state=0)

In [ ]:
#imputing 0 into NaN values for logistic regression after OHE
X_cancel_train = X_cancel_train.fillna(0)
X_cancel_test = X_cancel_test.fillna(0)
y_cancel_train = y_cancel_train.fillna(0)
y_cancel_test = y_cancel_test.fillna(0)

In [ ]:
#imputing 0 into NaN values for logistic regression after OHE
X_delay_train = X_delay_train.fillna(0)
X_delay_test = X_delay_test.fillna(0)
y_delay_train = y_delay_train.fillna(0)
y_delay_test = y_delay_test.fillna(0)

In [ ]:
#checking shape of data
print('X_cancel_train shape: ',X_cancel_train.shape,'\n')
print('X_cancel_test shape: ',X_cancel_test.shape,'\n')
print('y_cancel_train shape: ',y_cancel_train.shape,'\n')
print('y_cancel_test shape: ',y_cancel_test.shape)

In [ ]:
#checking shape of data
print('X_delay_train shape: ',X_delay_train.shape,'\n')
print('X_delay_test shape: ',X_delay_test.shape,'\n')
print('y_delay_train shape: ',y_delay_train.shape,'\n')
print('y_delay_test shape: ',y_delay_test.shape)

***
# Modeling and Evaluation
***

### LGBM:
***

In [ ]:
#https://towardsdatascience.com/getting-the-most-out-of-scikit-learn-pipelines-c2afc4410f1a
#https://lightgbm.readthedocs.io/en/latest/Parameters.html#objective

# Define a pipeline to search for the best combination of PCA truncation

pca_lgbm = PCA(n_components = .95,
               svd_solver = 'auto')


In [ ]:
# Define a Standard Scaler to normalize inputs

scaler_lgbm = RobustScaler()

lr = 0.15
boost = 'dart' #'gbdt' 
obj = 'binary'
tl = 'voting' # 'data'
numiter = 1000
mbin = 300
nleaves = 42


In [ ]:
# Scale -> PCA -> LR 
lgbm = lgb.LGBMClassifier(learning_rate = lr,
                          boosting = boost,
                          objective = obj,
                          tree_learner = tl,
                          #num_iterations = numiter,
                          num_leaves = nleaves,
                          max_bin = mbin,
                          extra_trees = True)


In [ ]:
%%time
#start_lgbm = time.time()
pipe_lgbm = Pipeline(steps=[("scaler", scaler_lgbm),
                       ("pca", pca_lgbm),
                       ("LightGBM", lgbm)])

pipe_lgbm.fit(X_train,y_train)
#end_lgbm = time.time()

In [ ]:
#predict
y_pred_lgbm=pipe_lgbm.predict(X_test)

In [ ]:
accuracy_lgbm = metrics.accuracy_score(y_test,y_pred_lgbm)
precision_lgbm = metrics.precision_score(y_test, y_pred_lgbm)
recall_lgbm = metrics.recall_score(y_test, y_pred_lgbm)

In [ ]:
#print results 
print('____________Start LightGBM Base Model Results______________:')
print('Accuracy:\n',accuracy_lgbm,'\n')
print('Precision:\n',recall_lgbm,'\n')
print('Recall:\n',recall_lgbm,'\n')
print('____________End LightGBM Base Model Results________________')

***
## Random Forest:
***

In [ ]:
#Importing required libraries.
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
#Specififying Stratified Kfold for cv.
skfcv = StratifiedKFold(n_splits=10)

In [ ]:
#Creating our random forest pipeline.
rf = RandomForestClassifier(n_estimators=100, random_state=0)

In [ ]:
#Fitting random forest model on training data.
rf.fit(X_train, y_train)

In [ ]:
#Predicting on our test set.
y_pred_rf = rf.predict(X_test)

In [ ]:
#Code adapted from: https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652
rfc_cv_score = cross_val_score(rf, X_test, y_test, cv=skfcv, scoring='accuracy')


In [ ]:
#Code adapted from: https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred_rf))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_pred_rf))
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

# Gridsearch

#Specifying the number of trees in random forest:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 10)]
#Specifying criterion for each split:
criterion = ['gini','entropy','log_loss']
#Specifying the maximum number of levels in the tree:
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
#Specifying the number of features to consider at every split:
max_features = ['auto','sqrt', 'log2','none']
#Specifying the minimum number of samples required to split a node:
min_samples_split = [2, 5, 10]
#Specifying the minimum number of samples required at each leaf node:
min_samples_leaf = [1, 2, 4]
#Specifying the method of selecting samples for training each tree:
bootstrap = [True, False]
#Creating and printing my random grid with parameters defined above:

njobs = [-1]
verbo = [1]
param_grid= {'n_estimators': n_estimators,
               'criterion': criterion,
               'verbose': verbo,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'n_jobs': njobs
            }

print(param_grid)

#Base RF model used to tune:
rf = RandomForestClassifier(random_state=0)
rf_random = GridSearchCV(estimator = rf, param_grid = param_grid, cv = skfcv)
#Fiting the random search model on our training data:
rf_random.fit(X_train, y_train)
print(rf_random.best_estimator_)

In [ ]:
print(rf_random.best_estimator_)

# CatBoost

In [ ]:
df_air_catb = df_air.copy()

In [ ]:
#convert object fields and category fields 
categories = ['year','month','day','day_of_week','airline_cd','airline_name','flight_number','origin_airport','tail_number',
              'airport_origin','city_origin','state_origin','country_origin','destination_airport','airport_destination',
              'city_destination','state_destination','country_destination','cancellation_reason','is_delay']

df_air_catb[categories] = df_air_catb[categories].fillna('Unknown')

In [ ]:
df_air_catb[categories] = df_air_catb[categories].astype('category')

In [ ]:
df_air_catb.info(verbose = True)

In [ ]:
df_air_catb['tail_number'] = df_air_catb['tail_number'].astype('category')
df_air_catb['airport_origin'] = df_air_catb['airport_origin'].astype('category')

In [ ]:
#remove target variable and column target was created from training data set
target_cat_cols = ['is_delay','departure_delay']
feature_cat_cols = df_air.loc[:, ~df_air_catb.columns.isin(target_cat_cols)].columns
X_cat = df_air_catb[feature_cat_cols] # Features
y_cat = df_air_catb.is_delay # Target variable

In [ ]:
#split original dataset into features and target variable
X_cat_train,X_cat_test,y_cat_train,y_cat_test=train_test_split(X_cat,y_cat,test_size=0.20,random_state=0)

In [ ]:
#checking shape of data
print('X_cat_train shape: ',X_cat_train.shape,'\n')
print('X_cat_test shape: ',X_cat_test.shape,'\n')
print('y_cat_train shape: ',y_cat_train.shape,'\n')
print('y_cat_test shape: ',y_cat_test.shape)

In [ ]:
cat_feat = df_air_catb.select_dtypes(include=['category']).columns.tolist()

In [ ]:
%%time
#https://www.kaggle.com/code/mitribunskiy/tutorial-catboost-overview/notebook

seed = 0
params = {'loss_function':'Logloss', # objective function
          'eval_metric':'AUC', # metrics
          'verbose': 200, # output to stdout info about training process every 200 iterations
          'random_seed': seed,
          'learning_rate': .25,
          'cat_features': cat_feat
         }

cbc = CatBoostClassifier(**params)
cbc.fit(Xcat_train, ycat_train, # data to train on (required parameters, unless we provide X as a pool object, will be shown below)
          eval_set=(Xcat_test, ycat_test), # data to validate on
          use_best_model=True, # True if we don't want to save trees created after iteration with the best validation score
          plot=True # True for visualization of the training process (it is not shown in a published kernel - try executing this code)
         );

***
# Deployment
***

***
# Exceptional Work
***